In [ ]:
# important libraries
!pip install --quiet transformers
!pip install --quiet pytorch-lightning
!pip install --quiet torchmetrics

In [2]:
# setup g-drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# import warnings
# warnings.filterwarnings('ignore')

In [4]:
# check for gpu
!nvidia-smi

Mon Jan 17 09:10:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import pandas as pd
import numpy as np

from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl
import torchmetrics as torchmetrics
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, multilabel_confusion_matrix, f1_score

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

In [ ]:
# configuration
%matplotlib inline  
%config InlineBackend.figure_format='retina'

RANDOM_SEED = 42

sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

pl.seed_everything(RANDOM_SEED)

In [ ]:
# install data
!wget https://raw.githubusercontent.com/chanzuckerberg/DRSM-corpus/main/labeled_data_2022_01_03.tsv

In [9]:
# retrieve data
def get_data(path_to_file: str, test_size=0.15):
    # read tsv file
    df = pd.read_csv(path_to_file, sep="\t")
    df = df[['TRIMMED_TEXT', 'Correct_Label']]
    df.dropna(inplace=True) 

    # label columns
    LABEL_COLUMNS = df['Correct_Label'].unique().tolist()
    # one-hot encoding
    df = pd.get_dummies(df, columns=['Correct_Label'])
    # rename columns
    rename_dict = dict((item1, item2) for item1, item2 in zip(df.columns[1:].tolist(), LABEL_COLUMNS))
    df.rename(columns=rename_dict, inplace=True, errors="raise")
    # split dataset
    train_df, test_df = train_test_split(df, test_size=test_size)

    return train_df, test_df, LABEL_COLUMNS

In [10]:
train_df, test_df, LABEL_COLUMNS = get_data(path_to_file="/content/labeled_data_2022_01_03.tsv")

In [12]:
# initialize SPECTER
model_name = "dmis-lab/biobert-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/462 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [14]:
MAX_TOKEN_COUNT = 512
N_EPOCHS = 10
BATCH_SIZE = 4

In [15]:
class TopicAnnotationDataset(Dataset):

  def __init__(
    self,
    data: pd.DataFrame,
    tokenizer: AutoTokenizer,
    max_token_len: int = 512,
  ):
    self.tokenizer = tokenizer
    self.data = data
    self.max_token_len = max_token_len
    
  def __len__(self):
    return len(self.data)

  def __getitem__(self, index=int):
    
    # extract row using index
    data_row = self.data.iloc[index]

    # get data (text, labels)
    text = data_row['TRIMMED_TEXT']
    labels = data_row[LABEL_COLUMNS]

    # apply tokenization
    inputs = self.tokenizer.encode_plus(
        text, 
        max_length=self.max_token_len,
        padding="max_length", 
        truncation=True, 
        return_tensors="pt", 
    )

    return dict(
        text=text,
        input_ids=inputs["input_ids"].flatten(),
        attention_mask=inputs["attention_mask"].flatten(),
        labels=torch.FloatTensor(labels) 
    )  

In [16]:
train_dataset = TopicAnnotationDataset(
    train_df,
    tokenizer=tokenizer,
)

In [35]:
class TopicAnnotationDataModule(pl.LightningDataModule):

  def __init__(self, train_df, test_df, tokenizer, batch_size=8, max_token_len=128):
    
    super().__init__()
    self.batch_size = batch_size
    self.train_df = train_df
    self.test_df = test_df
    self.tokenizer = tokenizer
    self.max_token_len = max_token_len

  def setup(self, stage=None):
    
    # setup train data
    self.train_dataset =  TopicAnnotationDataset(
        data=self.train_df,
        tokenizer=self.tokenizer,
        max_token_len=self.max_token_len
    )

    # setup test data
    self.test_dataset = TopicAnnotationDataset(
        data=self.test_df,
        tokenizer=self.tokenizer,
        max_token_len=self.max_token_len
    )

  def train_dataloader(self):

    return DataLoader(
        self.train_dataset,
        batch_size=self.batch_size,
        shuffle=True,
        num_workers=8
    )

  def val_dataloader(self):

    return DataLoader(
        self.test_dataset,
        batch_size=self.batch_size,
        num_workers=8
    )

  def test_dataloader(self):
    
    return DataLoader(
        self.test_dataset,
        batch_size=self.batch_size,
        num_workers=8
    )

In [36]:
data_module = TopicAnnotationDataModule(
    train_df,
    test_df,
    tokenizer,
    batch_size=BATCH_SIZE,
    max_token_len=MAX_TOKEN_COUNT
)

In [19]:
# calculate class-wise weights
label_count = train_df[LABEL_COLUMNS].sum().to_dict()
count = list(label_count.values())
max_val = max(count)
class_weight = [max_val/val for val in count]
# transfer to gpu
class_weight = torch.tensor(class_weight, device="cuda")

In [48]:
class TopicAnnotationTagger(pl.LightningModule):

  def __init__(self, n_classes: int, n_training_steps=None, n_warmup_steps=None):
    
    super().__init__()
    self.bert = AutoModel.from_pretrained(model_name, return_dict=True)
    self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
    self.n_training_steps = n_training_steps
    self.n_warmup_steps = n_warmup_steps
    self.criterion = nn.BCELoss(weight=class_weight)

  def forward(self, input_ids, attention_mask, labels=None):

    output = self.bert(input_ids, attention_mask=attention_mask)
    output = self.classifier(output.pooler_output)
    output = torch.sigmoid(output)
    loss = 0
    if labels is not None:
      loss = self.criterion(output, labels)
    return loss, output

  def training_step(self, batch, batch_idx):

    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]

    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("train_loss", loss, prog_bar=True, logger=True, batch_size=BATCH_SIZE)
    return {"loss": loss, "predictions": outputs, "labels": labels}

  def validation_step(self, batch, batch_idx):

    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("val_loss", loss, prog_bar=True, logger=True, batch_size=BATCH_SIZE)
    return loss

  def test_step(self, batch, batch_idx):
    
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("test_loss", loss, prog_bar=True, logger=True, batch_size=BATCH_SIZE)
    return loss

  def training_epoch_end(self, outputs):

    labels = []
    predictions = []
    for output in outputs:
      for out_labels in output["labels"].detach().cpu():
        labels.append(out_labels)
      for out_predictions in output["predictions"].detach().cpu():
        predictions.append(out_predictions)

    labels = torch.stack(labels).int()
    predictions = torch.stack(predictions)

    for i, name in enumerate(LABEL_COLUMNS):
      auroc = torchmetrics.AUROC(num_classes=len(LABEL_COLUMNS))
      class_roc_auc = auroc(predictions[:, i], labels[:, i])
      self.logger.experiment.add_scalar(f"{name}_roc_auc/Train", class_roc_auc, self.current_epoch)

  def configure_optimizers(self):

    optimizer = AdamW(self.parameters(), lr=2e-5)

    scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps=self.n_warmup_steps,
      num_training_steps=self.n_training_steps
    )

    return dict(
        optimizer=optimizer,
      lr_scheduler=dict(
        scheduler=scheduler,
        interval='step'
      )
    )

In [49]:
# calculate warmup steps and total training steps
steps_per_epoch=len(train_df) // BATCH_SIZE
total_training_steps = steps_per_epoch * N_EPOCHS
warmup_steps = total_training_steps // 5
warmup_steps, total_training_steps

(3790, 18950)

----
#### Train

In [50]:
model = TopicAnnotationTagger(
    n_classes=len(LABEL_COLUMNS),
    n_warmup_steps=warmup_steps,
    n_training_steps=total_training_steps
)

In [24]:
# %cd '/content/drive/MyDrive/Nidhir_Akash_Biocreative/'

/content/drive/MyDrive/Nidhir_Akash_Biocreative


In [44]:
# set-up checkpoints annd logs directory
checkpoint_callback = ModelCheckpoint(
    dirpath="model_3_checkpoints",
    filename="model_3_checkpoints",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)
logger = TensorBoardLogger("model_3_logs", name="topic-annotations")

# early-stopping criterion
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)

In [46]:
# setup trainer and add training arguments
trainer = pl.Trainer(
    logger=logger,
    checkpoint_callback=True,
    callbacks=[checkpoint_callback, early_stopping_callback],
    max_epochs=N_EPOCHS,
    gpus=1,
    progress_bar_refresh_rate=30
)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:148: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=True)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=True)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=30)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, u

In [51]:
# train model
trainer.fit(model, data_module)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:470: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type      | Params
-----------------------------------------
0 | bert       | BertModel | 108 M 
1 | classifier | Linear    | 4.6 K 
2 | criterion  | BCELoss   | 0     
-----------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
433.260   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Global seed set to 42


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/optimization/closure.py:36: LightningDeprecationWarning: One of the returned values {'labels', 'predictions'} has a `grad_fn`. We will detach it automatically but this behaviour will change in v1.6. Please detach it manually: `return {'loss': ..., 'something': something.detach()}`
  f"One of the returned values {set(extra.keys())} has a `grad_fn`. We will detach it automatically"


...

Epoch 1, global step 5687: val_loss reached 0.72667 (best 0.72667), saving model to "/content/drive/MyDrive/Nidhir_Akash_Biocreative/model_3_checkpoints/model_3_checkpoints.ckpt" as top 1


Validating: 0it [00:00, ?it/s]

Epoch 2, global step 7583: val_loss was not in top 1


Validating: 0it [00:00, ?it/s]

Epoch 3, global step 9479: val_loss was not in top 1


-----
#### Evaluation

In [52]:
path_to_model_checkpoints = '/content/drive/MyDrive/Nidhir_Akash_Biocreative/model_3_checkpoints/model_3_checkpoints.ckpt'
trained_model = TopicAnnotationTagger.load_from_checkpoint(path_to_model_checkpoints, n_classes=6)
trained_model.freeze()

In [53]:
# transfer model to gpu if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
trained_model = trained_model.to(device)

# create test data
test_dataset = TopicAnnotationDataset(
    test_df,
    tokenizer,
    max_token_len=MAX_TOKEN_COUNT
)

In [54]:
predictions = []
labels = []

# generate predictions 
for item in tqdm(test_dataset):

  _, prediction = trained_model(
      item["input_ids"].unsqueeze(dim=0).to(device),
      item["attention_mask"].unsqueeze(dim=0).to(device),
  )
  predictions.append(prediction.flatten())
  labels.append(item["labels"].int())

predictions = torch.stack(predictions).detach().cpu()
labels = torch.stack(labels).detach().cpu()

  0%|          | 0/1338 [00:00<?, ?it/s]

In [55]:
# calculate accuracy
threshold = 0.5
f1_score(labels, predictions>threshold, average=None)

array([0.9220104 , 0.94469027, 0.        , 0.32258065, 0.8       ,
       0.88235294])

In [56]:
# AUROC
print("AUROC per tag")
for i, name in enumerate(LABEL_COLUMNS):
  auroc = torchmetrics.AUROC(num_classes=len(LABEL_COLUMNS))
  tag_auroc = auroc(predictions[:, i], labels[:, i])
  print(f"{name}: {tag_auroc}")

AUROC per tag
clinical characteristics or disease pathology: 0.9717940092086792
other: 0.9844029545783997
disease mechanism: 0.8841341733932495
therapeutics in the clinic: 0.8667154908180237
irrelevant: 0.9935858845710754
patient-based therapeutics: 0.9860029220581055


/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [57]:
# classification report
y_pred = predictions.numpy()
y_true = labels.numpy()

upper, lower = 1, 0

y_pred = np.where(y_pred > 0.5, upper, lower)

print(
    classification_report(
        y_true,
        y_pred,
        digits=4,
        target_names=LABEL_COLUMNS,
        zero_division=0
    )
)

                                               precision    recall  f1-score   support

clinical characteristics or disease pathology     0.9204    0.9236    0.9220       576
                                        other     0.9323    0.9574    0.9447       446
                            disease mechanism     0.0000    0.0000    0.0000        17
                   therapeutics in the clinic     1.0000    0.1923    0.3226        52
                                   irrelevant     0.8636    0.7451    0.8000        51
                   patient-based therapeutics     0.9270    0.8418    0.8824       196

                                    micro avg     0.9243    0.8759    0.8995      1338
                                    macro avg     0.7739    0.6100    0.6453      1338
                                 weighted avg     0.9146    0.8759    0.8841      1338
                                  samples avg     0.8752    0.8759    0.8754      1338

